In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [2]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [3]:
user_data = pd.read_sql('SELECT * FROM public.user_data', con = engine)
posts = pd.read_sql('SELECT * FROM public.post_text_df', con = engine)


In [ ]:
#feed_data = pd.read_sql('SELECT * FROM public.feed_data LIMIT 2000000', con = engine)
feed_like = pd.read_sql("""SELECT * FROM public.feed_data
                        limit 8000000""", con = engine)

In [ ]:
#3 человека ничего не оценили
print(feed_like.user_id.nunique())
print(user_data.user_id.nunique())
#print(feed_like[feed_like['user_id'] == 200])

17246
163205


In [9]:
#feed_data['target'].value_counts()
# оставим до поры....
feed_like = feed_like[feed_like['action'] == 'view']
feed_like

,timestamp,user_id,post_id,action,target
0,2021-10-21 19:03:11,79246,6648,view,0
1,2021-10-21 19:03:31,79246,1298,view,0
2,2021-10-21 19:05:37,79246,967,view,1
4,2021-10-21 19:08:01,79246,1661,view,0
5,2021-10-21 19:09:25,79246,3647,view,0
...,...,...,...,...,...
7999994,2021-11-17 16:51:12,150077,4009,view,0
7999995,2021-11-17 16:51:58,150077,5754,view,1
7999997,2021-11-17 16:54:07,150077,2636,view,0
7999998,2021-11-17 16:55:44,150077,2361,view,0


In [10]:
feed_like.target.value_counts()

target
0    6294895
1     852552
Name: count, dtype: int64

In [12]:
feed_like.post_id.min()

1

In [13]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download("stopwords") # поддерживает удаление стоп-слов
nltk.download('punkt') # делит текст на список предложений
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladimirkazakov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vladimirkazakov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vladimirkazakov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
# Лемматизация и стемминг
nltk.download('punkt_tab')

new_text = []
lemmatize = nltk.WordNetLemmatizer()
for i in posts.text:
    text = i.lower()
    text = re.sub("[^a-zA-Z]"," ", text)
    tokens = nltk.word_tokenize(text, language = "english")
    lemmatized_tokens = [lemmatize.lemmatize(word) for word in tokens]

    processed_text = ' '.join(lemmatized_tokens)
    new_text.append(processed_text)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/vladimirkazakov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [15]:
len(new_text)

7023

In [16]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

#posts = posts.dropna()
tfidf = TfidfVectorizer(stop_words='english')
train_tfidf = tfidf.fit_transform(new_text)

tfidf_df = pd.DataFrame(
    train_tfidf.toarray(),
    columns = tfidf.get_feature_names_out()
)


In [17]:
tfidf_df

,aa,aaa,aaaaah,aaaahhhhhhh,aaaggghhhhhhh,aaah,aac,aacl,aad,aadc,...,zyafqvi,zybc,zyfeepff,zyivyltmv,zylbkt,zyrsfn,zyy,zz,zztohapny,zzzzzzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Не центруем - матрица разряжена!
# Выбираем подход -> TruncatedSVD
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components = 100)
SVD_dataset = svd.fit_transform(tfidf_df)


In [ ]:
SVD_dataset = pd.DataFrame(SVD_dataset,
                           columns = [f'Компонента {i+1}' for i in range(SVD_dataset.shape[1])]
                          )

In [20]:
SVD_dataset

,Компонента 1,Компонента 2,Компонента 3,Компонента 4,Компонента 5,Компонента 6,Компонента 7,Компонента 8,Компонента 9,Компонента 10,...,Компонента 91,Компонента 92,Компонента 93,Компонента 94,Компонента 95,Компонента 96,Компонента 97,Компонента 98,Компонента 99,Компонента 100
0,0.116060,-0.186410,-0.049454,-0.000526,-0.101012,-0.133809,-0.202105,-0.065665,-0.073923,-0.106965,...,0.000313,-0.038108,-0.018725,-0.007628,-0.061584,0.004443,-0.034623,-0.018739,-0.033567,0.021570
1,0.144863,-0.213865,-0.057816,-0.049957,0.040706,-0.041323,-0.027854,0.008911,-0.021749,0.014682,...,-0.020554,0.040811,-0.025180,-0.007439,0.009892,0.012054,-0.031371,-0.046268,-0.043021,0.008408
2,0.124525,-0.147405,-0.033435,0.010893,-0.114624,-0.132848,-0.157768,-0.064739,-0.007802,-0.029390,...,-0.033472,0.034620,0.023558,0.009818,-0.011366,-0.002854,0.010094,-0.002634,-0.063665,-0.022565
3,0.099042,-0.150082,-0.023253,-0.003138,-0.075871,-0.127193,-0.110217,-0.029228,0.039352,0.046404,...,-0.003989,0.046915,-0.012014,-0.001384,0.049319,-0.017751,0.044567,-0.015547,-0.010442,-0.010421
4,0.064908,-0.077955,-0.022646,0.000432,-0.032226,-0.038090,-0.007138,0.005748,0.010516,0.034957,...,-0.045661,0.010505,-0.001591,-0.000405,-0.001114,0.020748,0.010800,-0.007568,-0.021621,-0.004185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,0.254660,0.124200,0.017228,-0.143191,-0.034005,0.014996,0.008833,-0.018280,0.056755,-0.043683,...,0.055271,-0.029625,0.025359,0.013953,-0.004037,0.021516,0.026534,-0.023153,-0.038156,0.004956
7019,0.209761,0.108771,0.014662,-0.092210,-0.014202,-0.046050,-0.001095,0.022621,-0.030199,0.000516,...,-0.021475,0.002565,-0.003714,-0.035966,-0.018376,-0.024487,-0.013974,-0.001210,0.020745,0.010574
7020,0.138492,0.056343,0.003408,0.115936,0.076048,-0.027880,0.013415,-0.041474,0.004164,0.002194,...,-0.029177,-0.015199,0.000564,0.018089,-0.011392,-0.029148,0.005413,-0.013646,0.001490,0.011800
7021,0.200977,0.030445,-0.010369,0.093321,0.052155,0.010091,-0.014534,0.010124,0.099706,-0.056337,...,0.011128,0.022718,-0.002781,0.032127,-0.036391,-0.000442,-0.022046,0.020471,0.008349,-0.004062


In [21]:
posts.isna().sum()

post_id    0
text       0
topic      0
dtype: int64

In [ ]:
posts = pd.concat((posts, pd.get_dummies(posts['topic'], dtype = int, drop_first=True)), axis=1)

In [ ]:
posts = posts.drop(['topic', 'text'], axis=1)

In [24]:
posts.head(3)

,post_id,covid,entertainment,movie,politics,sport,tech
0,1,0,0,0,0,0,0
1,2,0,0,0,0,0,0
2,3,0,0,0,0,0,0


In [25]:
preprocess_posts = pd.merge(
    SVD_dataset,
    posts,
    left_index=True,
    right_index=True,
    how = 'outer'
)

In [26]:
preprocess_posts = preprocess_posts.set_index('post_id', drop = True)
preprocess_posts

,Компонента 1,Компонента 2,Компонента 3,Компонента 4,Компонента 5,Компонента 6,Компонента 7,Компонента 8,Компонента 9,Компонента 10,...,Компонента 97,Компонента 98,Компонента 99,Компонента 100,covid,entertainment,movie,politics,sport,tech
post_id,,,,,,,,,,,,,,,,,,,,,
1,0.116060,-0.186410,-0.049454,-0.000526,-0.101012,-0.133809,-0.202105,-0.065665,-0.073923,-0.106965,...,-0.034623,-0.018739,-0.033567,0.021570,0,0,0,0,0,0
2,0.144863,-0.213865,-0.057816,-0.049957,0.040706,-0.041323,-0.027854,0.008911,-0.021749,0.014682,...,-0.031371,-0.046268,-0.043021,0.008408,0,0,0,0,0,0
3,0.124525,-0.147405,-0.033435,0.010893,-0.114624,-0.132848,-0.157768,-0.064739,-0.007802,-0.029390,...,0.010094,-0.002634,-0.063665,-0.022565,0,0,0,0,0,0
4,0.099042,-0.150082,-0.023253,-0.003138,-0.075871,-0.127193,-0.110217,-0.029228,0.039352,0.046404,...,0.044567,-0.015547,-0.010442,-0.010421,0,0,0,0,0,0
5,0.064908,-0.077955,-0.022646,0.000432,-0.032226,-0.038090,-0.007138,0.005748,0.010516,0.034957,...,0.010800,-0.007568,-0.021621,-0.004185,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,0.254660,0.124200,0.017228,-0.143191,-0.034005,0.014996,0.008833,-0.018280,0.056755,-0.043683,...,0.026534,-0.023153,-0.038156,0.004956,0,0,1,0,0,0
7316,0.209761,0.108771,0.014662,-0.092210,-0.014202,-0.046050,-0.001095,0.022621,-0.030199,0.000516,...,-0.013974,-0.001210,0.020745,0.010574,0,0,1,0,0,0
7317,0.138492,0.056343,0.003408,0.115936,0.076048,-0.027880,0.013415,-0.041474,0.004164,0.002194,...,0.005413,-0.013646,0.001490,0.011800,0,0,1,0,0,0


In [ ]:
# Записал локально обработанную табл posts -> preprocess_posts
#preprocess_posts.to_csv('preprocess_posts.csv', sep=',', encoding='utf-8')
# Записал в СУБД Karpov courses
preprocess_posts.to_sql('preprocess_posts', con = engine, if_exists='replace')

8

In [27]:

user_data.head(3)

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads


In [ ]:
categorical_features = user_data[user_data.select_dtypes(include=['object']).columns]

In [29]:
#categorical_features.os.unique()
#categorical_features.source.unique()
print(f'Колонки типа "object": {categorical_features.columns}')
print("Columns in user_data:", user_data.columns.tolist())
print("Columns in feed_like:", feed_like.columns.tolist())

Колонки типа "object": Index(['country', 'city', 'os', 'source'], dtype='object')
Columns in user_data: ['user_id', 'gender', 'age', 'country', 'city', 'exp_group', 'os', 'source']
Columns in feed_like: ['timestamp', 'user_id', 'post_id', 'action', 'target']


In [ ]:
result_2 = pd.merge(
    user_data,
    feed_like,
    left_on='user_id',
    right_on = 'user_id',
    how = 'inner')

In [ ]:
result_2

,user_id,gender,age,country,city,exp_group,os,source,timestamp,post_id,action,target
0,1943,1,26,Russia,Yelets,0,Android,ads,2021-11-07 17:41:56,4306,view,0
1,1943,1,26,Russia,Yelets,0,Android,ads,2021-11-07 17:43:58,6722,view,0
2,1943,1,26,Russia,Yelets,0,Android,ads,2021-11-07 17:45:08,4797,view,0
3,1943,1,26,Russia,Yelets,0,Android,ads,2021-11-07 17:45:24,6077,view,0
4,1943,1,26,Russia,Yelets,0,Android,ads,2021-11-07 17:45:42,5808,view,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7147442,163628,1,37,Russia,Lipetsk,4,iOS,organic,2021-11-17 14:40:20,3335,view,0
7147443,163628,1,37,Russia,Lipetsk,4,iOS,organic,2021-11-17 14:41:46,3238,view,0
7147444,163628,1,37,Russia,Lipetsk,4,iOS,organic,2021-11-17 14:42:51,507,view,0
7147445,163628,1,37,Russia,Lipetsk,4,iOS,organic,2021-11-17 14:43:19,4925,view,0


In [32]:
# categorical_features.corr() -> only numbers!
# Пишем сначала правило: для country | city and os | source
import numpy as np
C = 0.006
for col in categorical_features.columns:
    if categorical_features[col].nunique() <= 2:
        dummies = pd.get_dummies(result_2[col], prefix=col, dtype = int, drop_first=True)
        result_2 = pd.concat([result_2, dummies], axis=1)
        result_2 = result_2.drop(col, axis=1)
    else:
        means = result_2.groupby(col)['target'].mean()
        noisy_means = means + C * np.random.randn(len(means))
        mapping_dict = noisy_means.to_dict()
        result_2[col] = result_2[col].map(mapping_dict)
        

In [34]:
result_2 = result_2.drop(['action'], axis = 1)

In [ ]:
from datetime import datetime

result_2['day'] = result_2['timestamp'].dt.dayofweek
result_2['month'] = result_2['timestamp'].dt.month
result_2['hour'] = result_2['timestamp'].dt.hour

In [37]:
result_2.head(2)

,user_id,gender,age,country,city,exp_group,timestamp,post_id,target,os_iOS,source_organic,day,month,hour
0,1943,1,26,0.105758,0.071714,0,2021-11-07 17:41:56,4306,0,0,0,6,11,17
1,1943,1,26,0.105758,0.071714,0,2021-11-07 17:43:58,6722,0,0,0,6,11,17


In [ ]:
# Теперь еще раз пробуем с корреляцией
def get_redundant_pairs(result_2):
    pairs_to_drop = set()
    cols = result_2.columns
    for i in range(0, result_2.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(result_2, n=5):
    au_corr = result_2.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(result_2)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(result_2, 15))

Top Absolute Correlations
timestamp  month             0.942326
user_id    source_organic    0.833291
country    city              0.482353
city       target            0.135054
country    target            0.064594
day        month             0.057844
target     month             0.057360
timestamp  target            0.054874
           day               0.048679
age        post_id           0.032946
gender     target            0.027037
city       exp_group         0.025055
target     hour              0.019303
gender     city              0.014545
country    os_iOS            0.013605
dtype: float64


In [ ]:
# merge -> preprocess_posts and result_2
df = pd.merge(
    preprocess_posts,
    result_2,
    left_index=True,
    right_on = 'post_id',
    how = 'inner')

In [40]:
df.user_id.count()

np.int64(7147447)

In [44]:
df.target.value_counts()

target
0    1571020
1     214490
Name: count, dtype: int64

In [41]:
df = df.set_index(['user_id', 'post_id'])

In [42]:
df.head(20)

Компонента 1  Компонента 2  Компонента 3  Компонента 4  \
user_id post_id                                                           
1953    1             0.11606      -0.18641     -0.049454     -0.000526   
1961    1             0.11606      -0.18641     -0.049454     -0.000526   
        1             0.11606      -0.18641     -0.049454     -0.000526   
1963    1             0.11606      -0.18641     -0.049454     -0.000526   
2032    1             0.11606      -0.18641     -0.049454     -0.000526   
2062    1             0.11606      -0.18641     -0.049454     -0.000526   
2066    1             0.11606      -0.18641     -0.049454     -0.000526   
2143    1             0.11606      -0.18641     -0.049454     -0.000526   
2168    1             0.11606      -0.18641     -0.049454     -0.000526   
2187    1             0.11606      -0.18641     -0.049454     -0.000526   
2205    1             0.11606      -0.18641     -0.049454     -0.000526   
2259    1             0.11606      -0.18641     -0.049454     -0.000526   
2267    1             0.11606      -0.18641     -0.049454     -0.000526   
2280    1             0.11606      -0.18641     -0.049454     -0.000526   
2288    1             0.11606      -0.18641     -0.049454     -0.000526   
2292    1             0.11606      -0.18641     -0.049454     -0.000526   
2311    1             0.11606      -0.18641     -0.049454     -0.000526   
2323    1             0.11606      -0.18641     -0.049454     -0.000526   
2328    1             0.11606      -0.18641     -0.049454     -0.000526   
2336    1             0.11606      -0.18641     -0.049454     -0.000526   

                 Компонента 5  Компонента 6  Компонента 7  Компонента 8  \
user_id post_id                                                           
1953    1           -0.101012     -0.133809     -0.202105     -0.065665   
1961    1           -0.101012     -0.133809     -0.202105     -0.065665   
        1           -0.101012     -0.133809     -0.202105     -0.065665   
1963    1           -0.101012     -0.133809     -0.202105     -0.065665   
2032    1           -0.101012     -0.133809     -0.202105     -0.065665   
2062    1           -0.101012     -0.133809     -0.202105     -0.065665   
2066    1           -0.101012     -0.133809     -0.202105     -0.065665   
2143    1           -0.101012     -0.133809     -0.202105     -0.065665   
2168    1           -0.101012     -0.133809     -0.202105     -0.065665   
2187    1           -0.101012     -0.133809     -0.202105     -0.065665   
2205    1           -0.101012     -0.133809     -0.202105     -0.065665   
2259    1           -0.101012     -0.133809     -0.202105     -0.065665   
2267    1           -0.101012     -0.133809     -0.202105     -0.065665   
2280    1           -0.101012     -0.133809     -0.202105     -0.065665   
2288    1           -0.101012     -0.133809     -0.202105     -0.065665   
2292    1           -0.101012     -0.133809     -0.202105     -0.065665   
2311    1           -0.101012     -0.133809     -0.202105     -0.065665   
2323    1           -0.101012     -0.133809     -0.202105     -0.065665   
2328    1           -0.101012     -0.133809     -0.202105     -0.065665   
2336    1           -0.101012     -0.133809     -0.202105     -0.065665   

                 Компонента 9  Компонента 10  ...   country      city  \
user_id post_id                               ...                       
1953    1           -0.073923      -0.106965  ...  0.105758  0.158928   
1961    1           -0.073923      -0.106965  ...  0.105758  0.094438   
        1           -0.073923      -0.106965  ...  0.105758  0.094438   
1963    1           -0.073923      -0.106965  ...  0.176379  0.176323   
2032    1           -0.073923      -0.106965  ...  0.105758  0.065226   
2062    1           -0.073923      -0.106965  ...  0.105758  0.079863   
2066    1           -0.073923      -0.106965  ...  0.105758  0.100578   
2143    1           -0.073923      -0.106965  ...  0.10575

In [81]:
print(df['timestamp'].max())
print(df['timestamp'].min())

2021-12-29 23:51:06
2021-10-01 06:05:25


In [43]:
df_train = df[df['timestamp'] <= '2021-12-15']
df_test = df[df['timestamp'] > '2021-12-15']

df_train = df_train.drop(['timestamp'],axis=1)
df_test = df_test.drop(['timestamp'],axis=1)

X_train = df_train.drop(['target'], axis=1)
Y_train = df_train['target']

X_test = df_test.drop(['target'], axis=1)
Y_test = df_test['target']


In [59]:
def train_neural_network(X_train, X_test, Y_train, Y_test):
    # Нормализация данных - ОБУЧАЕМ scaler ТОЛЬКО на тренировочных данных
    scaler = StandardScaler()
    X_scaled_train = scaler.fit_transform(X_train)  # fit_transform только на train
    X_scaled_test = scaler.transform(X_test)        # transform на test
    
    # Создаем validation set из тренировочных данных (последние 20%)
    train_size = int(0.8 * len(X_scaled_train))
    X_tr = X_scaled_train[:train_size]
    X_val = X_scaled_train[train_size:]
    
    # Преобразуем y в numpy arrays
    y_tr = Y_train[:train_size]
    y_val = Y_train[train_size:]
    
    # Создание модели
    input_dim = X_train.shape[1]
    model = MLPClassifier(input_dim=input_dim)
    
    # Обучение
    trainer = NeuralNetworkTrainer(model)
    trainer.train(X_tr, y_tr, X_val, y_val, scaler, input_dim)
    
    return trainer, scaler


In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm.auto import tqdm

# Архитектура нейросети
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], dropout_rates=[0.3, 0.3, 0.2]):
        super(MLPClassifier, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for i, (hidden_dim, dropout_rate) in enumerate(zip(hidden_dims, dropout_rates)):
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_dim = hidden_dim
        
        self.layers = nn.Sequential(*layers)
        self.output = nn.Linear(prev_dim, 1)
        
    def forward(self, x):
        x = self.layers(x)
        x = self.output(x)
        return torch.sigmoid(x)

# Класс для обучения
class NeuralNetworkTrainer:
    def __init__(self, model, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model.to(device)
        self.device = device
        
    def train(self, X_train, y_train, X_val, y_val, scaler, input_dim, epochs=3, batch_size=32, lr=0.001):
        # Преобразование в тензоры
        X_train_tensor = torch.FloatTensor(X_train).to(self.device)
        y_train_tensor = torch.FloatTensor(y_train.values if hasattr(y_train, 'values') else y_train).to(self.device)
        X_val_tensor = torch.FloatTensor(X_val).to(self.device)
        y_val_tensor = torch.FloatTensor(y_val.values if hasattr(y_val, 'values') else y_val).to(self.device)
        
        # Создание DataLoader
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        # Оптимизатор и функция потерь
        optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=1e-5)
        criterion = nn.BCELoss()
        
        # Планировщик learning rate
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
        
        best_val_loss = float('inf')
        patience = 10
        patience_counter = 0

        epoch_pbar = tqdm(range(epochs), desc="Training Progress", unit="epoch")
        
        for epoch in epoch_pbar:
            # Обучение
            self.model.train()
            train_loss = 0.0
            
            # Прогресс-бар для батчей внутри эпохи
            batch_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False, unit="batch")
            
            for batch_X, batch_y in batch_pbar:
                optimizer.zero_grad()
                
                outputs = self.model(batch_X)
                loss = criterion(outputs.squeeze(), batch_y)
                
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()

                # Обновляем описание для батча
                batch_pbar.set_postfix({
                    'Batch Loss': f'{loss.item():.4f}',
                    'Avg Train Loss': f'{train_loss/(batch_pbar.n+1):.4f}'
                })
            
            batch_pbar.close()
            
            # Валидация
            self.model.eval()
            with torch.no_grad():
                val_outputs = self.model(X_val_tensor)
                val_loss = criterion(val_outputs.squeeze(), y_val_tensor)
                
            scheduler.step(val_loss)
            
            # Ранняя остановка
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                torch.save(self.model.state_dict(), 'best_model.pth')
            else:
                patience_counter += 1
                
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch}: Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}")
        
        # Загрузка лучшей модели перед сохранением complete_model
        self.model.load_state_dict(torch.load('best_model.pth'))
        
        # Сохранение полной модели со scaler
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'scaler': scaler,
            'input_dim': input_dim,
            'hidden_dims': [256, 128, 64],
            'dropout_rates': [0.3, 0.3, 0.2]
        }, 'complete_model.pth')
        
        epoch_pbar.close()
        print(f"Training completed! Best val loss: {best_val_loss:.4f}")
        print("Complete model saved as 'complete_model.pth'")
        
    def predict_proba(self, X):
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X).to(self.device)
            predictions = self.model(X_tensor)
            return predictions.cpu().numpy()

# Функция для обучения и сохранения
def train_and_save_model(X_train, X_test, Y_train, Y_test):
    # Нормализация данных
    scaler = StandardScaler()
    X_scaled_train = scaler.fit_transform(X_train)
    X_scaled_test = scaler.transform(X_test)
    
    # Создаем validation set из тренировочных данных
    train_size = int(0.8 * len(X_scaled_train))
    X_tr = X_scaled_train[:train_size]
    X_val = X_scaled_train[train_size:]
    y_tr = Y_train[:train_size]
    y_val = Y_train[train_size:]
    
    # Создание модели
    input_dim = X_train.shape[1]
    model = MLPClassifier(input_dim=input_dim)
    
    print(f"Начало обучения нейросети:")
    print(f"- Размерность входа: {input_dim}")
    print(f"- Тренировочные данные: {X_tr.shape[0]} samples")
    print(f"- Валидационные данные: {X_val.shape[0]} samples")
    
    # Обучение с передачей scaler и input_dim
    trainer = NeuralNetworkTrainer(model)
    trainer.train(X_tr, y_tr, X_val, y_val, scaler, input_dim)
    
    return trainer

# Использование
# trainer = train_and_save_model(X_train, X_test, Y_train, Y_test)

In [61]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

trainer, scaler = train_neural_network(X_train, X_test, Y_train, Y_test)
# Предсказание на тестовой выборке
test_predictions = trainer.predict_proba(scaler.transform(X_test))

Training Progress:   0%|          | 0/3 [00:00<?, ?epoch/s]

Epoch 1/3:   0%|          | 0/149167 [00:00<?, ?batch/s]

Epoch 0: Train Loss: 0.3422, Val Loss: 0.3445


Epoch 2/3:   0%|          | 0/149167 [00:00<?, ?batch/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

Epoch 3/3:   0%|          | 0/149167 [00:00<?, ?batch/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Training completed! Best val loss: 0.3433
Complete model saved as 'complete_model.pth'


In [63]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

test_pred_binary = (test_predictions > 0.5).astype(int).flatten()

print(f"\n📊 Результаты на тестовой выборке:")
print(f"✅ Accuracy: {accuracy_score(Y_test.values, test_pred_binary):.4f}")
print(f"📈 ROC-AUC: {roc_auc_score(Y_test.values, test_predictions.flatten()):.4f}")
print(f"\n📋 Classification Report:")
print(classification_report(Y_test.values, test_pred_binary))


📊 Результаты на тестовой выборке:
✅ Accuracy: 0.8619
📈 ROC-AUC: 0.6507

📋 Classification Report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.93   1017685
           1       0.00      0.00      0.00    163119

    accuracy                           0.86   1180804
   macro avg       0.43      0.50      0.46   1180804
weighted avg       0.74      0.86      0.80   1180804



/Users/vladimirkazakov/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/vladimirkazakov/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/vladimirkazakov/opt/anaconda3/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{me

In [94]:
# Смотрим итоговые колонки -- || --
#Y_train.head(3)
Y_test.value_counts()

target
0    254162
1     40392
Name: count, dtype: int64

In [87]:
from catboost import CatBoostClassifier

### Your code is here
catboost = CatBoostClassifier(random_state = 1, max_depth=5, learning_rate=0.2)
catboost.fit(X_train, Y_train)
y_pred = catboost.predict(X_test)


0:	learn: 0.5531070	total: 187ms	remaining: 3m 6s
1:	learn: 0.4712348	total: 314ms	remaining: 2m 36s
2:	learn: 0.4240476	total: 451ms	remaining: 2m 29s
3:	learn: 0.3950682	total: 582ms	remaining: 2m 24s
4:	learn: 0.3769954	total: 694ms	remaining: 2m 18s
5:	learn: 0.3661004	total: 801ms	remaining: 2m 12s
6:	learn: 0.3595304	total: 932ms	remaining: 2m 12s
7:	learn: 0.3548688	total: 1.04s	remaining: 2m 8s
8:	learn: 0.3521083	total: 1.15s	remaining: 2m 6s
9:	learn: 0.3502234	total: 1.28s	remaining: 2m 6s
10:	learn: 0.3481147	total: 1.41s	remaining: 2m 6s
11:	learn: 0.3473026	total: 1.54s	remaining: 2m 7s
12:	learn: 0.3466945	total: 1.66s	remaining: 2m 5s
13:	learn: 0.3457109	total: 1.77s	remaining: 2m 4s
14:	learn: 0.3453159	total: 1.89s	remaining: 2m 4s
15:	learn: 0.3443301	total: 2.03s	remaining: 2m 4s
16:	learn: 0.3440290	total: 2.16s	remaining: 2m 4s
17:	learn: 0.3438313	total: 2.25s	remaining: 2m 2s
18:	learn: 0.3437284	total: 2.38s	remaining: 2m 3s
19:	learn: 0.3435645	total: 2.5s	re

In [90]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import roc_auc_score

accuracy = accuracy_score(Y_test, y_pred)
print(f'Accuracy на тесте: {accuracy:.4f}')

f1 = f1_score(Y_test, y_pred, average='weighted')
print(f'F1 на тесте: {f1:.4f}')

print(f"Качество на трейне roc_auc: {roc_auc_score(Y_train, catboost.predict_proba(X_train)[:,1])}")
print(f"Качество на тесте roc_auc: {roc_auc_score(Y_test, catboost.predict_proba(X_test)[:,1])}")
#print(classification_report(y_test, y_pred))

Accuracy на тесте: 0.8627
F1 на тесте: 0.7997
Качество на трейне roc_auc: 0.7313915385426885
Качество на тесте roc_auc: 0.6829554744528323


In [91]:
# Сохраняем модель с весами

catboost.save_model('catboost.cbm')

''' пример загрузки - >
loaded_model = CatBoostClassifier()
loaded_model.load_model('catboost.cbm')
'''

" пример загрузки - >\nloaded_model = CatBoostClassifier()\nloaded_model.load_model('catboost.cbm')\n"

In [24]:
# проверка загрузки preprocess_posts

output_postgres = pd.read_sql('SELECT * FROM public.preprocess_posts', con = engine)
output_postgres.head(3)

,post_id,Компонента 1,Компонента 2,Компонента 3,Компонента 4,Компонента 5,Компонента 6,Компонента 7,Компонента 8,Компонента 9,...,Компонента 97,Компонента 98,Компонента 99,Компонента 100,covid,entertainment,movie,politics,sport,tech
0,1,0.116060,-0.186410,-0.049454,-0.000525,-0.101009,-0.133808,-0.202089,-0.065688,-0.073895,...,-0.010917,-0.025841,0.028009,-0.051560,0,0,0,0,0,0
1,2,0.144863,-0.213865,-0.057816,-0.049957,0.040709,-0.041332,-0.027845,0.008899,-0.021784,...,-0.023653,-0.093682,-0.024064,-0.051820,0,0,0,0,0,0
2,3,0.124525,-0.147405,-0.033435,0.010892,-0.114626,-0.132842,-0.157777,-0.064754,-0.007903,...,0.009781,0.013621,0.048129,-0.032911,0,0,0,0,0,0
